In [121]:
import os
import os.path
import time
from datetime import datetime
import numpy as np
import math
import matplotlib.pyplot as plt 
import scipy
import scipy.io as spio
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import svm
import urllib
import urllib.request
from urllib.request import urlopen
%matplotlib inline

In [135]:
## Import data
training_filename = 'dtrain123.dat'
test_filename = 'dtest123.dat'
filename = 'zipcombo.dat'
data = np.loadtxt(filename)
## define feature vector x and true value y
x = data[:,1:]
y = data[:,0]

In [123]:
def construct_table(training_set_errors,test_set_errors):
    train_mean=np.mean(training_set_errors,axis=1)
    train_std=np.std(training_set_errors,axis=1)
    test_mean=np.mean(test_set_errors,axis=1)
    test_std=np.std(test_set_errors,axis=1)

    mean_std = []
    for i in range(len(train_mean)):
        data_t = []
        colomn_1 = "{0:.4f} +- {1:.4f}".format(train_mean[i]*100,train_std[i]*100)
        data_t.append(colomn_1)
        colomn_2 = "{0:.4f} +- {1:.4f}".format(test_mean[i]*100,test_std[i]*100)    
        data_t.append(colomn_2)
        mean_std.append(data_t)
    return mean_std

In [124]:
def basic_results_svm(x,y,cost_range,kernel_choice,runs):
    """
    Performs 'runs' iterations over the data set and each iteration:
    - for every kernel choice found in kernels array
        - for every value of C found in cost_range array
            Trains an SVM predictor with this combination on a random 80% portion of the data.
            Evaluates the prediction error as observed in the remaining 20% of the data.
    :param cost_range: an array of C values
    :param kernels: an array of kernel choices (must be a supported value in scikit-learn's SVC implementation)
    :param runs: the number of iterations to perform
    :return: two arrays: the error rates as observed in the training set and test set respectively.
    """
    training_set_errors = np.zeros((len(cost_range), runs))
    test_set_errors = np.zeros((len(cost_range), runs))
    
    for c in range(len(cost_range)):
        cost = cost_range[c]
        for i in range(runs):
            print("Now doing run ", i+1, "/", runs, " for ",", c=",cost,".........", end='\r')
            X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

            svm_classifier = svm.SVC(C=cost, kernel='poly', gamma='scale')
            svm_classifier.fit(X_train, y_train)

            test_labels = svm_classifier.predict(X_test)
            test_errors = sum(y_test != test_labels) / len(y_test)

            train_labels = svm_classifier.predict(X_train)
            train_errors = sum(y_train != train_labels) / len(y_train)

            training_set_errors[c, i] = train_errors
            test_set_errors[c, i] = test_errors

    return training_set_errors, test_set_errors

## Poly kernel and C

In [125]:
cost_range = [ 0.01, 0.1, 10, 100, 1000]
startTime = datetime.now()
svm_training_set_errors_poly, svm_test_set_errors_poly = basic_results_svm(x,y,cost_range,'poly',runs)
time_svm_poly_basic = datetime.now() - startTime
print("Time taken: ", time_svm_poly_basic)

Time taken:  0:25:49.845641  , c= 1000 .........


In [147]:
mean_std=construct_table(svm_training_set_errors_poly,svm_test_set_errors_poly)
pd.DataFrame(data=mean_std,index=cost_range,columns=['train_mean_std(%)','test_mean_std(%)'])

,train_mean_std(%),test_mean_std(%)
0.01,8.8088 +- 0.1653,9.1371 +- 0.6358
0.10,2.3602 +- 0.0904,3.4167 +- 0.3811
10.00,0.0208 +- 0.0067,2.2070 +- 0.3411
100.00,0.0087 +- 0.0064,2.2339 +- 0.2140
1000.00,0.0000 +- 0.0000,2.2608 +- 0.3342


## Gaussian Kernel of C

In [140]:
cost_range = [ 0.01, 0.1, 10, 100, 1000]
startTime = datetime.now()
svm_training_set_errors_rbf, svm_test_set_errors_rbf = basic_results_svm(x,y,cost_range,'rbf',runs)
time_svm_rbf_basic = datetime.now() - startTime
print("Time taken: ", time_svm_rbf_basic)

Time taken:  0:26:11.290748  , c= 1000 .........


In [148]:
Gaussian_mean_std=construct_table(svm_training_set_errors_rbf,svm_test_set_errors_rbf)
pd.DataFrame(data=Gaussian_mean_std,index=cost_range,columns=['train_mean_std(%)','test_mean_std(%)'])

,train_mean_std(%),test_mean_std(%)
0.01,8.7308 +- 0.1567,9.2903 +- 0.5797
0.10,2.3165 +- 0.0675,3.6720 +- 0.3925
10.00,0.0195 +- 0.0079,2.0645 +- 0.2907
100.00,0.0087 +- 0.0064,2.1102 +- 0.3274
1000.00,0.0000 +- 0.0000,2.3495 +- 0.2423


## Gamma of Gaussian Kernel

In [142]:
def basic_results_svm_2(x,y,Gamma,kernel,runs):
    
    G_training_set_errors = np.zeros((len(Gamma), runs))
    G_test_set_errors = np.zeros((len(Gamma), runs))

    for coefficient in range(len(Gamma)):
        g = Gamma[coefficient]
        for i in range(runs):
            print("Now doing run ", i+1, "/", runs, " for gamma=", g,".........", end='\r')
            X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

            svm_classifier = svm.SVC(C=1,kernel='rbf',gamma =g)
            svm_classifier.fit(X_train, y_train)

            train_labels = svm_classifier.predict(X_train)
            train_errors = sum(y_train != train_labels) / y_train.shape[0]

            test_labels = svm_classifier.predict(X_test)
            test_errors = sum(y_test != test_labels) / y_test.shape[0]


            G_training_set_errors[coefficient, i] = train_errors / y_train.shape[0]
            G_test_set_errors[coefficient, i] = test_errors / y_test.shape[0]
    
    return training_set_errors, test_set_errors

In [143]:
Gamma = np.array([0.01, 0.1, 10, 100, 1000])
startTime = datetime.now()
G_training_set_errors, G_test_set_errors=basic_results_svm_2(x,y,Gamma,'rbf',runs)
time_svm_cost_basic = datetime.now() - startTime
print("\nTime taken: ", time_svm_cost_basic)

Now doing run  20 / 20  for gamma= 1000.0 .........
Time taken:  1:21:54.261029


In [144]:
Gmean_std=construct_table(G_training_set_errors,G_test_set_errors)
pd.DataFrame(data=Gmean_std,index=Gamma,columns=['train_mean_std(%)','test_mean_std(%)'])

,train_mean_std(%),test_mean_std(%)
0.01,21.5858 +- 0.3486,22.0887 +- 0.8792
0.10,4.5335 +- 0.1084,5.0941 +- 0.4055
10.00,0.0309 +- 0.0121,2.2258 +- 0.2437
100.00,0.0128 +- 0.0029,2.3522 +- 0.2767
1000.00,0.0000 +- 0.0000,2.2742 +- 0.2845


## Cross validation for cost in 2 kernels

In [130]:
def cross_validation_svm(x,y,c,kernel_choice,k):
    """
    This function performs a k-fold cross validation on X, using a kernel of "kernel_choice" with parameter d.
    :param X: the observations array
    :param y: the labels vector
    :param kernel_choice: Depending on the kernel choice, can be {'Polynomial', 'Gaussian'}
    :param d: the parameter of the kernel
    :param k: the number of splits, i.e. the k parameter in k-fold Cross Validation
    :return: the mean of test error across the k runs of the CV process and its standard deviation
    """
    kf = KFold(n_splits=k, shuffle=True)
    mistake_arr = np.zeros(k)
    i = 0
    
    for train_index, cv_index in kf.split(x):
        # Spit the matrix using the indices gained by the CV method and construct X and Y arrays
        X_train = x[train_index]
        X_cv = x[cv_index]
        y_train = y[train_index]
        y_cv = y[cv_index]
    
        # We are only interested in the alphas and not the MSE on the training set
        
        svm_classifier = svm.SVC(C=c, kernel=kernel_choice, gamma='scale')
        svm_classifier.fit(X_train, y_train)
        predicted_labels = svm_classifier.predict(X_cv)
        
        mistakes = sum(y_cv != predicted_labels)
        mistake_arr[i] = mistakes / len(y_cv)
        i += 1
        
    return mistake_arr.mean()

In [131]:
def cv_process_svm(c_arr, runs, kernel_choice,x,y):
    """
    This function performs 5-fold cross validation, multiple times (specified by runs argument) across the different
    values of d specified in d_arr using the kernel specified in kernel_choice
    :param d_arr: an array of d values
    :param runs: The number of runs to repeat the CV process
    :param kernel_choice: Depending on the kernel choice, can be {'Polynomial', 'Gaussian'}
    :param calculate_confusions: Whether or not to also calculate confusions on the test set
    :return: the array of d_stars, the test_errors and the confusions found
    """
    c_stars = np.zeros(runs)
    test_errors = np.zeros(runs)
    for run in range(runs):
        single_confusion_mtx = np.zeros(shape = (10,10))
        # In each run we will iterate through the d array and use all possible values of d
        # Allocate 80/20 percent for training and test set
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

        CV_means = np.zeros(len(c_arr))
        for i in range(len(c_arr)):
            print("Now doing run ", run+1, "/", runs, " for d=", c_arr[i], ".........", end='\r')
            mistake = cross_validation_svm(X_train, y_train,c_arr[i],kernel_choice, k=5)
            CV_means[i] = mistake
            
        # Train in whole 80% now with c_star
        c_stars[run] = c_arr[CV_means.argmin()]
        # Train on whole of 80%
        svm_classifier = svm.SVC(C=CV_means.argmin(), kernel=kernel_choice, gamma='scale')
        svm_classifier.fit(X_train, y_train)
        predicted_labels = svm_classifier.predict(X_test)
        
        test_error = sum(y_test != predicted_labels) / len(y_test)
        test_errors[run] = test_error
    return c_stars,test_errors

## In Polynomial

In [146]:
runs = 20
c_arr = np.array([0.01,0.1, 10, 100, 1000])
startTime = datetime.now()
c_stars_array, test_errors_array = cv_process_svm(c_arr, runs,'poly',x,y)
time_CV_p_SVM = datetime.now() - startTime
print("Time taken: ", time_CV_p_SVM)
print("Mean c*: ", c_stars_array.mean(), " with std: ", np.std(c_stars_array))
print("Mean test error(%): ", test_errors_array.mean()*100, " with std(%): ", np.std(test_errors_array)*100)

Time taken:  0:52:12.543600 d= 1000.0 .........
Mean c*:  253.0  with std:  375.5276288104512
Mean test error(%):  2.1881720430107525  with std(%):  0.2783783541075187


## In Gaussian

In [139]:
runs = 20
c_arr = np.array([0.01,0.1, 10, 100, 1000])
startTime = datetime.now()
c_stars_array, test_errors_array = cv_process_svm(c_arr, runs,'rbf',x,y)
time_CV_G_SVM = datetime.now() - startTime
print("Time taken: ", time_CV_G_SVM)
print("Mean c*: ", c_stars_array.mean(), " with std: ", np.std(c_stars_array))
print("Mean test error(%): ", test_errors_array.mean()*100, " with std(%): ", np.std(test_errors_array)*100)

Time taken:  1:10:14.348494 d= 1000.0 .........
Mean c*:  100.0  with std:  211.06870919205434
Mean test error(%):  0.022016129032258063  with std(%):  0.002887627351578855
